# Explore, Clean, Organize the Ingredients!

In [1]:
import pandas as pd
import sqlite3 as sl3

In [2]:
dbfile = "recipes.db"

###Connecto to our SQLite3 DB File

In [3]:
conn = sl3.connect(dbfile)
cur = conn.cursor()

In [4]:
# Take a peek at the existing tables
cur.execute("SELECT name, sql FROM sqlite_master WHERE type='table'")
for table, sql in cur.fetchall():
    print table, sql

recipes CREATE TABLE recipes(
		recipe_id INTEGER, 
		search_query TEXT,
		url TEXT,
		website TEXT,
		timestamp TEXT,
		average_rating REAL,
		number_ratings INTEGER,
		yield_quantity REAL,
		yield_units TEXT,
		yield_type TEXT,
		PRIMARY KEY (recipe_id,url)
		)
ingredients CREATE TABLE ingredients(
		recipe_id INTEGER,
                ingredient_id INTEGER,
		description TEXT,
		amount REAL
		)


## Load 'description' into a DataFrame

In [5]:
# Read in 'ingredients' table
query = "SELECT recipe_id, description, amount FROM ingredients"
ingr_df = pd.read_sql(query, conn)
ingr_df.head()

,recipe_id,description,amount
0,0,shredded sharp Cheddar cheese,113.0
1,0,hummus,224.0
2,0,multigrain bread,208.0
3,0,"Granny Smith apples, thinly sliced",276.0
4,1,"garbanzo beans, drained",539.6


In [6]:
# How many do we have?
print len(ingr_df), 'entries'
print len(ingr_df.description.unique()), 'distinct ingredients'

842 entries
362 distinct ingredients


In [7]:
# Any other immediate insights?
ingr_df.describe()

,recipe_id,amount
count,842.000000,842.000000
mean,50.338480,97.628090
std,28.168521,174.339946
min,0.000000,0.000000
25%,27.000000,3.000000
50%,51.000000,24.695375
75%,75.000000,103.800000
max,99.000000,1230.000000


In [8]:
# A curiosity, the 'min' above is zero.
# What ingredient has a zero amount?!
ingr_df.query('amount == 0')

,recipe_id,description,amount
172,23,salt to taste,0
181,23,ground black pepper to taste,0
452,54,black pepper to taste,0
495,60,,0
497,60,salt and pepper to taste,0
548,66,,0
571,67,salt and pepper to taste,0
697,82,ground black pepper to taste,0
699,82,salt to taste,0
772,90,"lemons, zested and juiced",0


### We note already:
* This is only 11/800 or so ingredient entries, so not a big issue
* There are some NULL entries. Perhaps they indicate a small bug in the scraping?
* Some spices/ingredients may not be given an amount, but should not be ignored

In [9]:
# There's nothing we can do about the NULL description items
# Drop those entries
ingr_df = ingr_df[ingr_df['description']!='']

In [10]:
# Create a new field for whatever our filtered out description is
if 'new_description' not in ingr_df.columns.values:
    ingr_df['new_description'] = ""
# Do the same for the proportion field
if 'proportion' not in ingr_df.columns.values:
    ingr_df['proportion'] = 0.0
ingr_df.head()

,recipe_id,description,amount,new_description,proportion
0,0,shredded sharp Cheddar cheese,113.0,,0
1,0,hummus,224.0,,0
2,0,multigrain bread,208.0,,0
3,0,"Granny Smith apples, thinly sliced",276.0,,0
4,1,"garbanzo beans, drained",539.6,,0


In [11]:
# Get the usable fields from our 'recipes' table
query = "SELECT recipe_id, average_rating, number_ratings FROM recipes"
rec_df = pd.read_sql(query, conn)
rec_df.head()

,recipe_id,average_rating,number_ratings
0,0,4.319149,47
1,1,3.388889,18
2,2,4.657534,584
3,3,4.437500,16
4,4,4.165680,169


In [12]:
# Easy stuff first. Let's populate the proposrtion field!
for r_id in rec_df['recipe_id']:
    # Get the ingredient table entries for each recipe
    subset_df = ingr_df.query('recipe_id == @r_id').copy()
    # Get the total weight
    total_wt = subset_df['amount'].sum()
    # Scale the proportion column as amount / total weight
    subset_df.loc[:,'proportion'] = subset_df['amount'] / total_wt
    ingr_df.update(subset_df, join='left')

In [13]:
ingr_df.head()

,recipe_id,description,amount,new_description,proportion
0,0,shredded sharp Cheddar cheese,113.0,,0.137637
1,0,hummus,224.0,,0.272838
2,0,multigrain bread,208.0,,0.253350
3,0,"Granny Smith apples, thinly sliced",276.0,,0.336175
4,1,"garbanzo beans, drained",539.6,,0.680350


## Piece of cake, right?
* Time for the nastier bits now

In [14]:
counter = ingr_df['description'].value_counts()

In [15]:
print counter[0:30]

olive oil                                   43
lemon juice                                 40
tahini                                      38
ground cumin                                34
salt                                        31
garbanzo beans, drained                     30
garlic, minced                              21
garlic                                      16
paprika                                     15
water                                       13
fresh lemon juice                           13
cayenne pepper                              12
salt and ground black pepper to taste       10
extra-virgin olive oil                       9
salt and pepper to taste                     8
ground black pepper                          7
salt to taste                                7
dried oregano                                7
garbanzo beans, drained, liquid reserved     7
lemon, juiced                                7
crumbled feta cheese                         7
garlic, crush

##We have our work cut out for us...
I invoke the power of FuzzyWuzzy! 

You may have to install this with 'sudo pip install fuzzywuzzy' or 'sudo python -m pip install fuzzywuzzy'
(you may also want to check that you have 'python-levenshtein' and 'difflib' installed.

FuzzyWuzzy is good for fuzzily comparing two strings and returning a score. It also has a great module called 'process' that has the ability to make the best guess given a string and a pre-defined list of values. For the first go-around, I think this should suit our immediate needs.

###Strategy:
1. Go through by hand and identify major ingredients. I'm a human. I'm the best at that. Go humans!
2. Put all these into a list.
3. Go through each description, and have FuzzyWuzzy match it to an item from that list.
4. Populate 'new description' with that value
5. Evaluate how well this worked

In [16]:
from fuzzywuzzy import process

In [17]:
INGREDIENTS = set(["olive oil", "olives", "lemon juice", "tahini", 
                   "cumin", "salt", "garbonzo beans", "garlic",
                   "paprika", "water", "cayenne pepper", "black pepper",
                   "feta cheese", "oregano", "basil", "sesame seeds",
                   "parsley", "coriander", "curry powder", "balsamic vinegar",
                   "soy sauce", "onion powder", "yogurt", "bell pepper",
                   "avocado", "sweet potato", "black beans",
                   "chili powder", "pine nuts", "cream cheese", "cilantro",
                   "lemon zest", "pumpkin", "maple syrup", "parmesan cheese",
                   "sweet chili sauce", "sun-dried tomatoes", "peanut butter",
                   "white beans", "chickpeas", "white pepper", "cinnamon",
                   "tomato", "jalapeno", "soybeans", "dijon mustard", "hummus", 
                   "cheddar cheese", "artichoke hearts", "canola oil",
                   "red onion", "prosciutto", "canola oil", "mozzarella cheese",
                   "black olives", "allspice", "thyme", "chipotle pepper",
                   "ranch", "Italian dressing", "bacon", "zucchini", "rosemary",
                   "spinach", "tuna", "baking soda", "cucumber", "kalamata olives",
                   "green olives", "celery seed", "walnuts", "manzanilla olives",
                   "sesame seed oil", "monterey jack cheese", "red wine vinegar",
                   "beets", "yeast", "hot sauce", "crushed red pepper", "eggplant",
                   "marjoram", "ground beef", "ground lamb", "lettuce", "cashews",
                   "butter", "milk", "ginger", "lemon grass", "arugula", "flour",
                   "fennel seed", "egg", "coleslaw", "green onion", "vanilla extract",
                   "sage", "lemon pepper", "lime juice", "apples", "oatmeal",
                   "portabello mushroom", "flax seed", "apple sauce", "cottage cheese",
                   "tortilla", "chicken breast", "curry paste"])

In [18]:
MIN_SCORE = 75
def ingr_best_guess(x):
    # The extractOne gives a best guess based on the choices, along with a
    #    score based on how well it matches.
    # We arbitrarily set the 
    best_guess, score = process.extractOne(x['description'], INGREDIENTS)
    if score>MIN_SCORE:
        return best_guess
    else:
        return ""

In [19]:
ingr_df['new_description'] = ingr_df.apply(ingr_best_guess, axis=1)

In [20]:
ingr_df.head(20)

,recipe_id,description,amount,new_description,proportion
0,0,shredded sharp Cheddar cheese,113.000000,cheddar cheese,0.137637
1,0,hummus,224.000000,hummus,0.272838
2,0,multigrain bread,208.000000,,0.253350
3,0,"Granny Smith apples, thinly sliced",276.000000,apples,0.336175
4,1,"garbanzo beans, drained",539.600000,garbonzo beans,0.680350
5,1,olive oil,27.000000,olive oil,0.034043
6,1,creamy peanut butter,32.521010,peanut butter,0.041004
7,1,lemon juice,61.000000,lemon juice,0.076911
8,1,garlic,9.000000,garlic,0.011348
9,1,diced silken tofu,124.000000,,0.156344


## Looks good so far!
* I only went through the top occurrences to get started.
* Let's look at ones that had no good match and clean up the rest

In [21]:
ingr_df.query("new_description == ''")

,recipe_id,description,amount,new_description,proportion
2,0,multigrain bread,208.000000,,0.253350
9,1,diced silken tofu,124.000000,,0.156344
112,15,"onions, thinly sliced",908.000000,,0.196379
113,15,pieces lavash or other flatbread,1130.000000,,0.244393
116,15,brown sugar,40.704850,,0.008804
117,15,"Granny Smith apples - peeled, cored and cut in...",414.000000,,0.089539
124,16,"banana peppers, drained",453.600000,,0.375403
175,23,tofu,224.000000,,0.167133
177,23,tamari,18.000000,,0.013430
211,27,"albacore tuna in water, drained and flaked",168.000000,,0.765902


In [22]:
# Add some more entries based on the ones above
INGREDIENTS.update(set(["bread", "pita bread", "flatbread", "tofu", "broth", "bay leaf",
                        "serrano peppers", "onion", "hard-boiled egg", "broccoli", "corn",
                        "dill", "cabbage", "white sugar", "honey", "nutmeg", "raisins",
                        "cocounut oil", "brown sugar", "banana peppers", "pepporoncini peppers",
                        "tamari", "coconut", "pizza crust", "sandwich wrap", "wasabi",
                        "mayonnaise", "orange juice"]))

In [23]:
ingr_df['new_description'] = ingr_df.apply(ingr_best_guess, axis=1)
ingr_df.query("new_description == ''")

,recipe_id,description,amount,new_description,proportion
117,15,"Granny Smith apples - peeled, cored and cut in...",414,,0.089539
211,27,"albacore tuna in water, drained and flaked",168,,0.765902


### Instead of adding overly specific ingredient descriptions, let's try just easing up the minimum score threshhold

In [24]:
MIN_SCORE=55
ingr_df['new_description'] = ingr_df.apply(ingr_best_guess, axis=1)
ingr_df.query("new_description == ''")

,recipe_id,description,amount,new_description,proportion


# Let's inspect how good of a job this did!

In [25]:
ingr_df.head(20)

,recipe_id,description,amount,new_description,proportion
0,0,shredded sharp Cheddar cheese,113.000000,cheddar cheese,0.137637
1,0,hummus,224.000000,hummus,0.272838
2,0,multigrain bread,208.000000,bread,0.253350
3,0,"Granny Smith apples, thinly sliced",276.000000,apples,0.336175
4,1,"garbanzo beans, drained",539.600000,garbonzo beans,0.680350
5,1,olive oil,27.000000,olive oil,0.034043
6,1,creamy peanut butter,32.521010,peanut butter,0.041004
7,1,lemon juice,61.000000,lemon juice,0.076911
8,1,garlic,9.000000,garlic,0.011348
9,1,diced silken tofu,124.000000,tofu,0.156344


# Modifications
* After inspecting the master INGREDIENTS list, you can come up with rules
* Like, chickpeas should be set to garbonzo beans
* Or, any recipe for hummus that requires hummus is probably not a hummus recipe
* More mods can be added at this stage

In [26]:
# This gives the index values of entries that have 'chickpeas' as their 'new_description'
row_indexes = ingr_df[ ingr_df['new_description']=='chickpeas' ].index

In [27]:
# Set the value at these specific locations
ingr_df.loc[row_indexes,['new_description']] = 'garbonzo beans'

In [28]:
# Get recipe_id's that have 'hummus' as an ingredient
bad_recipe_id = ingr_df[ ingr_df['new_description'] == 'hummus' ]['recipe_id'].values
print bad_recipe_id

[  0.   8.  17.  27.  28.  40.  44.  53.  56.  61.  64.  70.  98.]


### You can do with these what you want. I'll just leave the information here.

# Okay, that's good for now. Let's write our work to the DB

In [29]:
ingr_df.to_sql('new_ingredients', conn, if_exists='replace')

# New 'base_ingredients' table
* Unique set of human-curated ingredients
* Some stats about their usage throughout existing recipes

In [30]:
# Get some metrics about each ingredient's usage
means = ingr_df[['new_description', 'proportion']].groupby('new_description').mean()
stddevs = ingr_df[['new_description', 'proportion']].groupby('new_description').std()
counts = ingr_df[['new_description', 'proportion']].groupby('new_description').count()

In [31]:
# Get our names of choice on the columns
means.rename(columns={'proportion': 'mean_proportion'}, inplace=True)
stddevs.rename(columns={'proportion': 'stddev_proportion'}, inplace=True)
counts.rename(columns={'proportion': 'counts'}, inplace=True)

In [32]:
# Stick em together into one frame
base_ingredients = pd.merge(means, stddevs, how='inner', left_index=True, right_index=True)
base_ingredients.head()

,mean_proportion,stddev_proportion
new_description,,
Italian dressing,0.100105,0.082766
allspice,0.000554,0.000400
apple sauce,0.089539,NaN
apples,0.215378,0.170833
artichoke hearts,0.256446,0.051010


In [33]:
base_ingredients = pd.merge(base_ingredients, counts, how='inner', left_index=True, right_index=True)
base_ingredients.head()

,mean_proportion,stddev_proportion,counts
new_description,,,
Italian dressing,0.100105,0.082766,4
allspice,0.000554,0.000400,2
apple sauce,0.089539,NaN,1
apples,0.215378,0.170833,2
artichoke hearts,0.256446,0.051010,3


In [35]:
# Move over the description field into a column and slap on a new unique ID
base_ingredients = base_ingredients.reset_index()
base_ingredients.head()

,new_description,mean_proportion,stddev_proportion,counts
0,Italian dressing,0.100105,0.082766,4
1,allspice,0.000554,0.000400,2
2,apple sauce,0.089539,NaN,1
3,apples,0.215378,0.170833,2
4,artichoke hearts,0.256446,0.051010,3


In [36]:
# Write this new table to our DB
base_ingredients.to_sql('base_ingredients', conn, if_exists='replace')
conn.close()

# Summary
This is just an exploratory look at how to handle this data. Clearly, any official handling will be done with a much more condensed python script. For now, things are looking pretty manageable. It's not a *good* solution to have a human-curated list of main ingredients, but it's the best and quickest thing I can come up with for now.

Note: All of these names and structures are not very well-thought out. I just wanted to show you guys around my meandering data cleaning methods. Most steps here can be highly compressed into much fewer steps, and the names can be a bit better chosen.